In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import json
import regex
import unidecode
from itertools import chain

In [2]:
# Load data
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/insiders.parquet")
df_in = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/outsiders.parquet")
df_ou = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/minors.parquet")
df_mi = pd.read_parquet(dir_df)

In [3]:
print(df_in.shape)
print(df_ou.shape)
print(df_mi.shape)

(519520, 195)
(236210, 43)
(1650869, 107)


In [4]:
# Load info from gencat
tend_cat = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")

C:\Users\josea\AppData\Local\Temp\ipykernel_39348\694994485.py:2: DtypeWarning: Columns (27,43,52,53,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  tend_cat = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")


In [5]:
# with open(r"C:\Users\josea\Downloads\genCat_Junio_2023.json", "r") as f:
#     gencat = pd.json_normalize(json.load(f))

# Aux functions

In [6]:
def unify_colname(col):
    return ".".join([el for el in col if el])


def evaluate(el):
    if not isinstance(el, str):
        return el
    if el[0] == "[" or el[0] == "'":
        return eval(el)
    return el


def fill_na(cell, fill=[]):
    """
    Fill elements in pd.DataFrame with `fill`.
    """
    if hasattr(cell, "__iter__"):
        if isinstance(cell, str):
            if cell == "nan":
                return fill
            return cell
        nas = []
        for el in cell:
            if el == "nan" or pd.isna(el):
                nas.append(True)
            else:
                nas.append(False)
        if all(nas):
            return fill
        return cell
    if pd.isna(cell):
        return fill
    return cell


def process_str(el: str):
    s = regex.sub(r"(^[^\w({[]*)|([^\w)}\]]*$)|([^\w:)(/-\s])", "", el).lower().strip()
    s = regex.sub(r"\s+", " ", s)
    return s


def process_cpv(el: str):
    return regex.sub(r"\D", "", el)

In [7]:
# Get general tender info (title, object, winner, etc)
df_in.columns = [unify_colname(c) for c in df_in.columns]
df_ou.columns = [unify_colname(c) for c in df_ou.columns]
df_mi.columns = [unify_colname(c) for c in df_mi.columns]

In [8]:
use_cols = [
    "id",
    "summary",
    "title",
    "updated",
    # "deleted_on",
    "ContractFolderStatus.ContractFolderID",
    "ContractFolderStatus.ContractFolderStatusCode",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyIdentification.ID",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyName.Name",
    "ContractFolderStatus.ProcurementProject.Name",
    "ContractFolderStatus.ProcurementProject.TypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TaxExclusiveAmount",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.DurationMeasure",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.StartDate",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.EndDate",
    # "ContractFolderStatus.TenderResult.ResultCode",
    # "ContractFolderStatus.TenderResult.ReceivedTenderQuantity",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
    "ContractFolderStatus.TenderResult.WinningParty.PartyName.Name",
    # "ContractFolderStatus.TenderResult.AwardedTenderedProject.LegalMonetaryTotal.TaxExclusiveAmount",
    # "ContractFolderStatus.TenderingProcess.ProcedureCode",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndDate",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndTime",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.Description",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod",
    # "ContractFolderStatus.ProcurementProject.SubTypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TotalAmount",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.IdentificationCode",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.Name",
    # "ContractFolderStatus.TenderResult.Description",
    # "ContractFolderStatus.TenderResult.AwardDate",
    # "ContractFolderStatus.TenderResult.StartDate",
    # "ContractFolderStatus.TenderResult.Contract.ID",
    # "ContractFolderStatus.TenderResult.Contract.IssueDate",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.CityName",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.PostalZone",
    "ContractFolderStatus.TenderingTerms.FundingProgramCode",
    "ContractFolderStatus.TenderingTerms.FundingProgram",
    "ContractFolderStatus.TenderResult.WinningParty.PartyLegalEntity.CompanyTypeCode",
]

In [9]:
rename_columns = {
    "title": "title",
    "summary": "summary",
    "ContractFolderStatus.ContractFolderID": "id",
    "ContractFolderStatus.ProcurementProject.Name": "project",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID": "winner",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode": "cpv",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity": "location",
    "denominacio": "title",
    "objecte_contracte": "summary",
    "codi_expedient": "id",
    "codi_cpv": "cpv",
}

# PLACE

In [10]:
df = pd.concat(
    [
        df_in[[c for c in use_cols if c in df_in.columns]],
        df_ou[[c for c in use_cols if c in df_ou.columns]],
        # df_mi[[c for c in use_cols if c in df_mi.columns]],
    ]
)
index_names = df.index.names
df.reset_index(inplace=True)
df["identifier"] = df[index_names].astype(str).agg("/".join, axis=1)
# df.drop(index_names, inplace=True, axis=1)
df.set_index("identifier", inplace=True)
df = df.applymap(fill_na, fill=np.nan)
df.shape

(755730, 24)

### Get relevant data

In [11]:
# Get only relevant data
tender_filt = (
    df[
        [
            "title",
            "summary",
            "ContractFolderStatus.ContractFolderID",
            # "ContractFolderStatus.ProcurementProject.Name",
            # "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
            "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
            "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
        ]
    ]
    .rename(columns=rename_columns)
    .dropna(how="all")
    .explode("cpv")
    # .explode("winner")
)
tender_filt["id_orig"] = tender_filt["id"]
tender_filt["id_norm"] = tender_filt["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
# tender_filt["winner"] = tender_filt["winner"].apply(evaluate)
tender_filt["cpv"] = tender_filt["cpv"].apply(evaluate)
tender_filt = tender_filt.explode("cpv")  # .explode("winner")
tender_filt["cpv"] = tender_filt["cpv"].astype(str).apply(process_cpv)
tender_filt["location"] = tender_filt["location"].apply(
    lambda x: unidecode.unidecode(x.lower()) if not pd.isna(x) else np.nan
)
tender_filt[[c for c in tender_filt.columns if c != "id"]] = (
    tender_filt[[c for c in tender_filt.columns if c != "id"]]
    .astype(str)
    .applymap(process_str)
)
tender_filt = tender_filt.replace({"": np.nan, "0": np.nan})
tender_filt["cpv_div"] = tender_filt["cpv"].apply(
    lambda x: x[:2] if not pd.isna(x) else np.nan
)
tender_filt = tender_filt.reset_index().reset_index()

### Get data from cat

In [12]:
# Filter using only those from Cat
agg_tender_orig = (
    tender_filt[
        tender_filt["location"].str.contains(
            "|".join(
                [
                    "nan",
                    "espana",
                    "cataluna",
                    "catalunya",
                    "barcelona",
                    "tarragona",
                    "girona",
                    "gerona",
                    "lleida",
                    "lerida",
                ]
            )
        )
    ]
    .drop("identifier", axis=1)
    .groupby(["id_orig"])
    .agg(list)
)
valid_agg_tender_orig = agg_tender_orig[
    agg_tender_orig["title"].apply(lambda x: len(set(x)) == 1)
]
valid_agg_tender_orig[valid_agg_tender_orig.columns[1:]] = valid_agg_tender_orig[
    valid_agg_tender_orig.columns[1:]
].applymap(lambda x: Counter(x).most_common()[0][0])
valid_agg_tender_orig = valid_agg_tender_orig.reset_index()
print(valid_agg_tender_orig.shape)
display(valid_agg_tender_orig.head())

C:\Users\josea\AppData\Local\Temp\ipykernel_39348\1730997490.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_agg_tender_orig[valid_agg_tender_orig.columns[1:]] = valid_agg_tender_orig[


(252111, 9)


,id_orig,index,title,summary,id,cpv,location,id_norm,cpv_div
0,(/2021,[997421],ampliació de lescola de freginals per a dispos...,id licitación: (/2021 órgano de contratación: ...,(/2021,452141000,nan,2021,45
1,(900243/22),[1007189],contracte de serveis per a laixecament topogrà...,id licitación: (900243/22) órgano de contratac...,(900243/22),713518100,nan,900243-22,71
2,(902472/21),[969935],contracte de serveis per a laixecament topogrà...,id licitación: (902472/21) órgano de contratac...,(902472/21),713518100,nan,902472-21,71
3,(902531/21),[969859],contracte de serveis per a laixecament topogrà...,id licitación: (902531/21) órgano de contratac...,(902531/21),713518100,nan,902531-21,71
4,(902766/21),[970783],contracte de serveis per a laixecament topogrà...,id licitación: (902766/21) órgano de contratac...,(902766/21),713518100,nan,902766-21,71


In [13]:
# Filter using only those from Cat
# Do the same, but in this case using the processed id
agg_tender_norm = (
    tender_filt[
        tender_filt["location"].str.contains(
            "|".join(
                [
                    "nan",
                    "espana",
                    "cataluna",
                    "catalunya",
                    "barcelona",
                    "tarragona",
                    "girona",
                    "gerona",
                    "lleida",
                    "lerida",
                ]
            )
        )
    ]
    .drop("identifier", axis=1)
    .groupby(["id_norm"])
    .agg(list)
)
valid_agg_tender_norm = agg_tender_norm[
    agg_tender_norm["title"].apply(lambda x: len(set(x)) == 1)
]
valid_agg_tender_norm[valid_agg_tender_norm.columns[1:]] = valid_agg_tender_norm[
    valid_agg_tender_norm.columns[1:]
].applymap(lambda x: Counter(x).most_common()[0][0])
valid_agg_tender_norm = valid_agg_tender_norm.reset_index()
print(valid_agg_tender_norm.shape)
display(valid_agg_tender_norm.head())

(245972, 9)


C:\Users\josea\AppData\Local\Temp\ipykernel_39348\705786846.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_agg_tender_norm[valid_agg_tender_norm.columns[1:]] = valid_agg_tender_norm[


,id_norm,index,title,summary,id,cpv,location,id_orig,cpv_div
0,0-2020-5-2020,[990577],rehabilitació de casa de poblet,id licitación: 0/2020-5/2020 órgano de contrat...,0/2020-5/2020,454541000,nan,0/2020-5/2020,45
1,00-0000,[884512],licitació de prova sobre digital 00/0000 - pro...,id licitación: 00/0000 órgano de contratación:...,00/0000,791000000,nan,00/0000,79
2,00-2019,[843883],comunicació de contractació anual programada (...,id licitación: 00/2019 órgano de contratación:...,00/2019,853200000,nan,00/2019,85
3,00-d-12-0000572,[996222],servicio de mantenimiento del sistema de alime...,id licitación: 00/d/12/0000572 órgano de contr...,00/D/12/0000572,505320000,nan,00/d/12/0000572,50
4,00-d-12-421,"[849661, 849662]",servicio de mantenimiento de la licencia dynat...,id licitación: 00/d/12/421 órgano de contratac...,00/D/12/421,48700000,nan,00/d/12/421,48


# GENCAT

In [14]:
# Clean tenders cat
tend_cat_filt = (
    tend_cat[
        [
            "denominacio",
            "objecte_contracte",
            "codi_expedient",
            "codi_cpv",
        ]
    ]
    .rename(columns=rename_columns)
    .astype(str)
)
tend_cat_filt["id_orig"] = tend_cat_filt["id"]
tend_cat_filt["id_norm"] = tend_cat_filt["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
tend_cat_filt["cpv"] = tend_cat_filt["cpv"].astype(str).apply(process_cpv)
tend_cat_filt[[c for c in tend_cat_filt if c != "id"]] = (
    tend_cat_filt[[c for c in tend_cat_filt if c != "id"]]
    .astype(str)
    .applymap(process_str)
)
tend_cat_filt = tend_cat_filt.replace({"": np.nan, "0": np.nan})
tend_cat_filt["cpv_div"] = tend_cat_filt["cpv"].apply(
    lambda x: x[:2] if not pd.isna(x) else np.nan
)
tend_cat_filt = tend_cat_filt.reset_index()

In [15]:
# tend_cat_filt[tend_cat_filt["id"]=="ccs-2022-6"]
# c = []
# for el in tend_cat[:3].apply(lambda x: Counter(x), axis=0):
#     c.append(3 not in el.values())
# tend_cat[:3][tend_cat.columns[c]]

In [16]:
# Aggregate by original ID
valid_tend_cat_orig = tend_cat_filt.groupby("id_orig").agg(list)
valid_tend_cat_orig = valid_tend_cat_orig[
    valid_tend_cat_orig["title"].apply(lambda x: len(set(x)) == 1)
]
valid_tend_cat_orig[valid_tend_cat_orig.columns[1:]] = valid_tend_cat_orig[
    valid_tend_cat_orig.columns[1:]
].applymap(lambda x: Counter(x).most_common()[0][0])
valid_tend_cat_orig = valid_tend_cat_orig.reset_index()
print(len(valid_tend_cat_orig))
display(valid_tend_cat_orig.head())

232124


,id_orig,index,title,summary,id,cpv,id_norm,cpv_div
0,(/2021,[9044],ampliacióde lescola de freginals i millora de ...,ampliació de lescola de freginals per a dispos...,(/2021,452141001,2021,45
1,(2) 49/2021/cmsubm2,[189719],compra del vestuari i dels equips de protecció...,compra del vestuari i dels equips de protecció...,(2) 49/2021/CMSUBM2,181100003,2-49-2021-cmsubm2,18
2,(2)120/2021/cmsubm1,[206137],subministrament de productes dhorticultura i v...,contracte de submnistrament de productes dhort...,(2)120/2021/CMSUBM1,031200008,2-120-2021-cmsubm1,03
3,(900243/22),[269614],contracte de serveis per a laixecament topogrà...,contracte de serveis per a laixecament topogrà...,(900243/22),713518104,900243-22,71
4,(902472/21),[187793],contracte de serveis per a laixecament topogrà...,contracte de serveis per a laixecament topogrà...,(902472/21),713518104,902472-21,71


In [17]:
# Aggregate by ID
valid_tend_cat_norm = tend_cat_filt.groupby("id_norm").agg(list)
valid_tend_cat_norm = valid_tend_cat_norm[
    valid_tend_cat_norm["title"].apply(lambda x: len(set(x)) == 1)
]
valid_tend_cat_norm[valid_tend_cat_norm.columns[1:]] = valid_tend_cat_norm[
    valid_tend_cat_norm.columns[1:]
].applymap(lambda x: Counter(x).most_common()[0][0])
valid_tend_cat_norm = valid_tend_cat_norm.reset_index()
print(len(valid_tend_cat_norm))
display(valid_tend_cat_norm.head())

227452


,id_norm,index,title,summary,id,cpv,id_orig,cpv_div
0,0-2020-5-2020,[153086],rehabilitació de casa de poblet,rehabilitació de casa de poblet,0/2020-5/2020,454541005,0/2020-5/2020,45
1,0-s-04-2023,[127873],gestió duna installació itinerant sobre la pro...,gestió duna installació itinerant que parli de...,0-S-04*/2023,794210001,0-s-04/2023,79
2,00-0000,[283367],licitació de prova sobre digital,licitació de prova sobre digital 00/0000 - pro...,00/0000,791000005,00/0000,79
3,000,[272105],concessió dús privatiu per a la installació de...,concessió dús privatiu per a la installació de...,000,NaN,000,NaN
4,000-expedient-2020-034,[182156],000 expedient 2020 034 - licitació de la contr...,en el marc duna creixent proliferació quant a ...,000 Expedient 2020 034,926220007,000 expedient 2020 034,92


# TODO
- Hacer match con budget

# Match PLACE & GENCAT

In [ ]:
# Match tenders orig
use_tend_agg_orig = valid_agg_tender_orig[["id_orig", "index", "title", "cpv_div"]]
use_tend_cat_orig = valid_tend_cat_orig[["id_orig", "index", "title", "cpv_div"]]

# Match tenders norm
use_tend_agg_norm = valid_agg_tender_norm[["id_norm", "index", "title", "cpv_div"]]
use_tend_cat_norm = valid_tend_cat_norm[["id_norm", "index", "title", "cpv_div"]]

### By ID

In [94]:
# Match tenders by ID orig
use_tend_agg_orig = valid_agg_tender_orig[["id_orig", "index", "title", "cpv_div"]]
use_tend_cat_orig = valid_tend_cat_orig[["id_orig", "index", "title", "cpv_div"]]
matched_id_orig = pd.merge(
    use_tend_agg_orig,
    use_tend_cat_orig,
    how="inner",
    left_on=["id_orig"],
    right_on=["id_orig"],
    suffixes=["_agg", "_cat"],
).reset_index(drop=True)
matched_id_orig["title"] = matched_id_orig[["title_agg", "title_cat"]].apply(
    lambda x: Counter(list(x)), axis=1
)
matched_id_orig["cpv_div"] = matched_id_orig[["cpv_div_agg", "cpv_div_cat"]].apply(
    lambda x: Counter(list(x)), axis=1
)
matched_id_orig = matched_id_orig[
    ["id_orig", "title", "cpv_div", "index_agg", "index_cat"]
]
idx_agg_orig = set(chain.from_iterable(matched_id_orig["index_agg"]))
idx_cat_orig = set(chain.from_iterable(matched_id_orig["index_cat"]))
print(f"Index agg: {len(idx_agg_orig)}")
print(f"Index cat: {len(idx_cat_orig)}")
print(f"Matched orig: {len(matched_id_orig)}")
display(matched_id_orig.head())

print("-" * 100)

# Match tenders by ID norm
use_tend_agg_norm = valid_agg_tender_norm[["id_norm", "index", "title", "cpv_div"]]
use_tend_cat_norm = valid_tend_cat_norm[["id_norm", "index", "title", "cpv_div"]]
matched_id_norm = pd.merge(
    use_tend_agg_norm,
    use_tend_cat_norm,
    how="inner",
    left_on=["id_norm"],
    right_on=["id_norm"],
    suffixes=["_agg", "_cat"],
).reset_index(drop=True)
matched_id_norm["title"] = matched_id_norm[["title_agg", "title_cat"]].apply(
    lambda x: Counter(list(x)), axis=1
)
matched_id_norm["cpv_div"] = matched_id_norm[["cpv_div_agg", "cpv_div_cat"]].apply(
    lambda x: Counter(list(x)), axis=1
)
matched_id_norm = matched_id_norm[
    ["id_norm", "title", "cpv_div", "index_agg", "index_cat"]
]
idx_agg_norm = set(chain.from_iterable(matched_id_norm["index_agg"]))
idx_cat_norm = set(chain.from_iterable(matched_id_norm["index_cat"]))
print(f"Index agg norm: {len(idx_agg_norm)}")
print(f"Index cat norm: {len(idx_cat_norm)}")
print(f"Matched norm: {len(matched_id_norm)}")
display(matched_id_norm.head())

Index agg: 97976
Index cat: 121326
Matched orig: 91798


,id_orig,title,cpv_div,index_agg,index_cat
0,(/2021,{'ampliació de lescola de freginals per a disp...,{'45': 2},[997421],[9044]
1,(900243/22),{'contracte de serveis per a laixecament topog...,{'71': 2},[1007189],[269614]
2,(902472/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[969935],[187793]
3,(902531/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[969859],[221217]
4,(902766/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[970783],[299956]


----------------------------------------------------------------------------------------------------
Index agg norm: 95084
Index cat norm: 117799
Matched norm: 88990


,id_norm,title,cpv_div,index_agg,index_cat
0,0-2020-5-2020,{'rehabilitació de casa de poblet': 2},{'45': 2},[990577],[153086]
1,00-0000,{'licitació de prova sobre digital 00/0000 - p...,{'79': 2},[884512],[283367]
2,000-expedient-2020-034,{'en el marc duna creixent proliferació quant ...,{'92': 2},[903809],[182156]
3,000-expedient-2021-266,{'el present expedient té per objecte el submi...,{'34': 2},[972632],[157778]
4,00000057-2022,{'renovació de la xarxa daigua potable i impla...,{'45': 2},[1007173],[189410]


#### Find unique values by IDs

In [91]:
# IDs in place (unique to orig):
idx_agg_un_orig = idx_agg_orig - idx_agg_norm
print(f"IDs in place (unique to orig): {len(idx_agg_un_orig)}")
# IDs in place (unique to norm):
idx_agg_un_norm = idx_agg_norm - idx_agg_orig
print(f"IDs in place (unique to norm): {len(idx_agg_un_norm)}")

# IDs in cat (unique to orig):
idx_cat_un_orig = idx_cat_orig - idx_cat_norm
print(f"IDs in place (unique to orig): {len(idx_cat_un_orig)}")
# IDs in cat (unique to norm):
idx_cat_un_norm = idx_cat_norm - idx_cat_orig
print(f"IDs in cat (unique to norm): {len(idx_cat_un_norm)}")


# matched_id_orig matched_id_norm

IDs in place (unique to orig): 3106
IDs in place (unique to norm): 214
IDs in place (unique to orig): 3783
IDs in cat (unique to norm): 256


In [92]:
# Indices to matched ids
agg2m_id_norm = {
    k: v
    for v, k in matched_id_norm["index_agg"].reset_index().explode("index_agg").values
}
cat2m_id_norm = {
    k: v
    for v, k in matched_id_norm["index_cat"].reset_index().explode("index_cat").values
}

agg2m_id_orig = {
    k: v
    for v, k in matched_id_orig["index_agg"].reset_index().explode("index_agg").values
}
cat2m_id_orig = {
    k: v
    for v, k in matched_id_orig["index_cat"].reset_index().explode("index_cat").values
}

##### Test unique values

In [36]:
# # Matches found with ID norm
# test = matched_id_norm.loc[
#     list(
#         set(
#             [agg2m_id_norm[i] for i in idx_agg_un_norm]
#             + [cat2m_id_norm[i] for i in idx_cat_un_norm]
#         )
#     )
# ]
# print(len(test))
# display(test.head())

In [37]:
# test_ids_agg = set(chain.from_iterable(test["index_agg"]))
# test_ids_cat = set(chain.from_iterable(test["index_cat"]))

# test_tend_agg_filt = tender_filt.loc[
#     list(test_ids_agg), ["id", "id_orig", "id_norm", "title"]
# ]
# test_tend_cat_filt = tend_cat_filt.loc[
#     list(test_ids_cat), ["id", "id_orig", "id_norm", "title"]
# ]

# symbols_agg = Counter("".join(test_tend_agg_filt["id_orig"].dropna().tolist()))
# symbols_cat = Counter("".join(test_tend_cat_filt["id_orig"].dropna().tolist()))
# print("".join(list(zip(*symbols_agg.most_common()))[0]))
# print("".join(list(zip(*symbols_cat.most_common()))[0]))

In [38]:
# display(
#     tender_filt.dropna(subset="id")[
#         tender_filt["id"]
#         .dropna()
#         .apply(lambda x: bool(regex.search(r"(cons_res_2022_0001)", x)))
#     ]
# )
# display(
#     tend_cat_filt.dropna(subset="id")[
#         tend_cat_filt["id"]
#         .dropna()
#         .apply(lambda x: bool(regex.search(r"(cons_res_2022_0001)", x)))
#     ]
# )

In [39]:
# print(
#     df.dropna(subset="ContractFolderStatus.ContractFolderID")[
#         df["ContractFolderStatus.ContractFolderID"]
#         .dropna()
#         .str.lower()
#         .apply(lambda x: bool(regex.search(r"(n801.*2022.*000552)", x)))
#     ][
#         [
#             "title",
#             "summary",
#             "updated",
#             "ContractFolderStatus.ContractFolderID",
#         ]
#     ].values
# )
# print(
#     tend_cat.dropna(subset="codi_expedient")[
#         tend_cat["codi_expedient"]
#         .dropna()
#         .str.lower()
#         .apply(lambda x: bool(regex.search(r"(n801.*2022.*000552)", x)))
#     ][
#         [
#             "denominacio",
#             "objecte_contracte",
#             "codi_expedient",
#         ]
#     ].values
# )

In [40]:
# cat_ids_symb = tend_cat_filt.loc[tend_cat_filt["id"].dropna().index]
# t = "_"
# potential_errors = cat_ids_symb.loc[
#     cat_ids_symb["id"].apply(lambda x: x.endswith(t) or x.startswith(t)),
#     "id",
# ].values
# # potential_errors

# els = list(set(potential_errors))
# print(els)

# els_all = r"\b|\b".join(els)
# els_all_alt = r"\b|\b".join([rf"{e.strip(t)}" for e in els])
# tf_with_symb = tender_filt.dropna(subset="id")[
#     tender_filt["id"].dropna().apply(lambda x: bool(regex.search(rf"({els_all})", x)))
# ]
# tf_without_symb = tender_filt.dropna(subset="id")[
#     tender_filt["id"]
#     .dropna()
#     .apply(lambda x: bool(regex.search(rf"(\b{els_all_alt}\b)", x)))
# ]

In [41]:
# els_all_alt = r"\b|\b".join([rf"{e.strip(t)}" for e in els])
# # els_all_alt = "\b"
# regex.search(rf"(\b{els_all_alt}\b)", "127")

In [ ]:
# Match tenders by ID norm
use_tend_agg_norm = valid_agg_tender_norm[["id_norm", "index", "title", "cpv_div"]]
use_tend_cat_norm = valid_tend_cat_norm[["id_norm", "index", "title", "cpv_div"]]
matched_id_norm = pd.merge(
    use_tend_agg_norm,
    use_tend_cat_norm,
    how="inner",
    left_on=["id_norm"],
    right_on=["id_norm"],
    suffixes=["_agg", "_cat"],
)
idx_agg_norm = set(chain.from_iterable(matched_id_norm["index_agg"]))
idx_cat_norm = set(chain.from_iterable(matched_id_norm["index_cat"]))
print(f"Index agg norm: {len(idx_agg_norm)}")
print(f"Index cat norm: {len(idx_cat_norm)}")
print(f"Matched norm: {len(matched_id_norm)}")
matched_id_norm.head()

### By title and CPV

In [169]:
# Match tenders by title and cpv orig
matched_title_orig = pd.merge(
    use_tend_agg_orig,
    use_tend_cat_orig,
    how="inner",
    left_on=[
        "title",
        "cpv_div",
    ],
    right_on=[
        "title",
        "cpv_div",
    ],
    suffixes=["_agg", "_cat"],
).reset_index(drop=True)

matched_title_orig = matched_title_orig[
    matched_title_orig["id_orig_agg"] != matched_title_orig["id_orig_cat"]
]

# Split to get those ids that only appear in agg or cat
m_x = matched_title_orig.loc[
    ~matched_title_orig["id_orig_agg"].isin(matched_title_orig["id_orig_cat"]),
    ["id_orig_agg", "index_agg", "title", "cpv_div"],
].rename({"id_orig_agg": "id_orig"}, axis=1)
m_y = matched_title_orig.loc[
    ~matched_title_orig["id_orig_cat"].isin(matched_title_orig["id_orig_agg"]),
    ["id_orig_cat", "index_cat", "title", "cpv_div"],
].rename({"id_orig_cat": "id_orig"}, axis=1)

# Group by id
matched_title_orig = (
    pd.concat([m_x, m_y])
    .groupby("id_orig")
    .agg(
        {
            "title": Counter,
            "cpv_div": Counter,
            "index_agg": sum,
            "index_cat": sum,
        }
    )
    .reset_index()
    .replace({0: None})
)
print(len(matched_title_orig))
print(sum(matched_title_orig["title"].apply(lambda x: sum(x.values()))))
display(matched_title_orig.head())

print("-" * 100)

matched_title_norm = pd.merge(
    use_tend_agg_norm,
    use_tend_cat_norm,
    how="inner",
    left_on=[
        "title",
        "cpv_div",
    ],
    right_on=[
        "title",
        "cpv_div",
    ],
    suffixes=["_agg", "_cat"],
).reset_index(drop=True)

matched_title_norm = matched_title_norm[
    matched_title_norm["id_norm_agg"] != matched_title_norm["id_norm_cat"]
]

# Split to get those ids that only appear in agg or cat
m_x = matched_title_norm.loc[
    ~matched_title_norm["id_norm_agg"].isin(matched_title_norm["id_norm_cat"]),
    ["id_norm_agg", "index_agg", "title", "cpv_div", "index_cat"],
].rename({"id_norm_agg": "id_norm"}, axis=1)
m_y = matched_title_norm.loc[
    ~matched_title_norm["id_norm_cat"].isin(matched_title_norm["id_norm_agg"]),
    ["id_norm_cat", "index_cat", "title", "cpv_div", "index_agg"],
].rename({"id_norm_cat": "id_norm"}, axis=1)

# Group by id
matched_title_norm = (
    pd.concat([m_x, m_y])
    .groupby("id_norm")
    .agg(
        {
            "title": Counter,
            "cpv_div": Counter,
            "index_agg": sum,
            "index_cat": sum,
        }
    )
    .reset_index()
    .replace({0: [None]})
)
print(len(matched_title_norm))
print(sum(matched_title_norm["title"].apply(lambda x: sum(x.values()))))
display(matched_title_norm.head())

2585
16325


,id_orig,title,cpv_div,index_agg,index_cat
0,001/21001877,{'gestió de la casa de laigua de trinitat nova...,{'98': 1},None,[268430]
1,001/21002039,{'servei de suport i assistència tècnica pel c...,{'71': 1},None,[237731]
2,001/21003081,{'subministrament de material de conservació i...,{'30': 1},[1016700],None
3,001/21004872,{'gestió i dinamització del centre cívic torre...,{'98': 1},None,[162624]
4,001/22,{'contractació dun servei que inclogui 3 perso...,{'72': 1},None,[212400]


----------------------------------------------------------------------------------------------------
2469
16185


,id_norm,title,cpv_div,index_agg,index_cat
0,001-21001877,{'gestió de la casa de laigua de trinitat nova...,{'98': 1},[1060594],[268430]
1,001-21002039,{'servei de suport i assistència tècnica pel c...,{'71': 1},[966599],[237731]
2,001-21003081,{'subministrament de material de conservació i...,{'30': 1},[1016700],"[192264, 192265]"
3,001-21004872,{'gestió i dinamització del centre cívic torre...,{'98': 1},[1016773],[162624]
4,001-22,{'contractació dun servei que inclogui 3 perso...,{'72': 1},[1069467],[212400]


### All (id, name, cpv)

In [182]:
a = pd.concat(
    [matched_id_orig, matched_title_orig],
)
a = a[a["id_orig"].duplicated(keep=False)].sort_values(by="id_orig")
a.dropna(subset="index_agg")[a["index_agg"].dropna().apply(len) > 1]

,id_orig,title,cpv_div,index_agg,index_cat
1036,0055/2018,{'lobjecte és la contractació de diverses pòli...,{'66': 2},"[848013, 848014, 848015]","[268331, 268332, 268333]"
34,0208051424 1/2019,{'subministrament de paper': 2},{'30': 2},"[865915, 865915]",None
49,08016781/2019/01,{'servei de neteja de linstitut esteve terrada...,{'90': 2},"[844125, 844125]",None
70,08030340/2019/01,{'servei de neteja de linstitut investigador b...,{'90': 2},"[840079, 840079]",None
136,08052876/2023/01,{'contracte del servei de neteja de linstitut ...,{'90': 2},"[1074635, 1074635]",None
...,...,...,...,...,...
2558,x2020037255,{'serveis postals de paqueteria i de missatger...,{'64': 1},"[932614, 932615]",None
90260,x2020037255,{'serveis postals de paqueteria i de missatger...,{'64': 2},"[932614, 932615]","[156945, 156946, 156947]"
90277,x2020059067,{'els serveis descrits al plec de prescripcion...,{'64': 2},"[960394, 960395]","[226540, 226541, 226542]"
2563,x2021000112,{'adquisició dun vehicle per als serveis tècni...,{'34': 2},"[964918, 964918]",None


In [183]:
2558
90260

,id_orig,title,cpv_div,index_agg,index_cat
494,001/21001877,{'gestió de lequipament públic i lorganització...,{'98': 2},[982625],[268430]
0,001/21001877,{'gestió de la casa de laigua de trinitat nova...,{'98': 1},None,[268430]
1,001/21002039,{'servei de suport i assistència tècnica pel c...,{'71': 1},None,[237731]
506,001/21002039,{'servei de suport i assistència tècnica pel c...,{'71': 2},[981114],[237731]
566,001/21003081,{'subministrament de material de conservació i...,"{'30': 1, nan: 1}",[1016700],[169383]
...,...,...,...,...,...
91692,x2022073121,{'la prestació del servei de teleassistència s...,{'85': 2},[1061868],[6366]
2583,x847/2019,{'servei de menjador de les escoles bressol mu...,{'55': 1},None,[219347]
91763,x847/2019,{'contractació del servei de menjador de les e...,{'55': 2},[894075],[219347]
91789,y8703,{'subministrament de comptadors electrònics da...,{'38': 2},[982888],[173060]


In [257]:
# Combine both
matched_final_orig = pd.merge(
    matched_id_orig,
    matched_title_orig,
    how="outer",
    left_on="id_orig",
    right_on="id_orig",
)

title = []
cpv_div = []
index_agg = []
index_cat = []
for tx, ty, cx, cy, iax, iay, icx, icy in matched_final_orig[
    [
        "title_x",
        "title_y",
        "cpv_div_x",
        "cpv_div_y",
        "index_agg_x",
        "index_agg_y",
        "index_cat_y",
        "index_cat_x",
    ]
].values:
    t = dict()
    c = dict()
    ia = []
    ic = []
    t.update(tx if not pd.isna(tx) else dict())
    t.update(ty if not pd.isna(ty) else dict())
    c.update(cx if not pd.isna(cx) else dict())
    c.update(cy if not pd.isna(cy) else dict())
    ia.extend(iax if isinstance(iax, list) else [])
    ia.extend(iay if isinstance(iay, list) else [])
    ic.extend(icx if isinstance(icx, list) else [])
    ic.extend(icy if isinstance(icy, list) else [])
    title.append(t)
    cpv_div.append(c)
    index_agg.append(ia)
    index_cat.append(ic)

matched_final_orig["title"] = title
matched_final_orig["cpv_div"] = cpv_div
matched_final_orig["index_agg"] = index_agg
matched_final_orig["index_cat"] = index_cat
matched_final_orig["index_agg"] = matched_final_orig["index_agg"].apply(
    lambda x: list(set(x))
)
matched_final_orig["index_cat"] = matched_final_orig["index_cat"].apply(
    lambda x: list(set(x))
)

matched_final_orig = matched_final_orig[
    ["id_orig", "title", "cpv_div", "index_agg", "index_cat"]
]
print(len(matched_final_orig))
# print(sum(matched_final_orig["title"].apply(lambda x: sum(x.values()))))
print(matched_final_orig[["index_agg", "index_cat"]].applymap(len).sum().to_dict())
display(matched_final_orig.head())

92274
{'index_agg': 98196, 'index_cat': 121648}


,id_orig,title,cpv_div,index_agg,index_cat
0,(/2021,{'ampliació de lescola de freginals per a disp...,{'45': 2},[997421],[9044]
1,(900243/22),{'contracte de serveis per a laixecament topog...,{'71': 2},[1007189],[269614]
2,(902472/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[969935],[187793]
3,(902531/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[969859],[221217]
4,(902766/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[970783],[299956]


In [258]:
# Combine both
matched_final_norm = pd.merge(
    matched_id_norm,
    matched_title_norm,
    how="outer",
    left_on="id_norm",
    right_on="id_norm",
)

title = []
cpv_div = []
index_agg = []
index_cat = []
for tx, ty, cx, cy, iax, iay, icx, icy in matched_final_norm[
    [
        "title_x",
        "title_y",
        "cpv_div_x",
        "cpv_div_y",
        "index_agg_x",
        "index_agg_y",
        "index_cat_y",
        "index_cat_x",
    ]
].values:
    t = dict()
    c = dict()
    ia = []
    ic = []
    t.update(tx if not pd.isna(tx) else dict())
    t.update(ty if not pd.isna(ty) else dict())
    c.update(cx if not pd.isna(cx) else dict())
    c.update(cy if not pd.isna(cy) else dict())
    ia.extend(iax if isinstance(iax, list) else [])
    ia.extend(iay if isinstance(iay, list) else [])
    ic.extend(icx if isinstance(icx, list) else [])
    ic.extend(icy if isinstance(icy, list) else [])
    title.append(t)
    cpv_div.append(c)
    index_agg.append(ia)
    index_cat.append(ic)

matched_final_norm["title"] = title
matched_final_norm["cpv_div"] = cpv_div
matched_final_norm["index_agg"] = index_agg
matched_final_norm["index_cat"] = index_cat
matched_final_norm["index_agg"] = matched_final_norm["index_agg"].apply(
    lambda x: list(set(x))
)
matched_final_norm["index_cat"] = matched_final_norm["index_cat"].apply(
    lambda x: list(set(x))
)

matched_final_orig = matched_final_norm[
    ["id_norm", "title", "cpv_div", "index_agg", "index_cat"]
]
print(len(matched_final_norm))
print(matched_final_norm[["index_agg", "index_cat"]].applymap(len).sum().to_dict())
display(matched_final_norm.head())

89449
{'index_agg': 96848, 'index_cat': 133159}


,id_norm,title_x,cpv_div_x,index_agg_x,index_cat_x,title_y,cpv_div_y,index_agg_y,index_cat_y,title,cpv_div,index_agg,index_cat
0,0-2020-5-2020,{'rehabilitació de casa de poblet': 2},{'45': 2},[990577],[153086],NaN,NaN,NaN,NaN,{'rehabilitació de casa de poblet': 2},{'45': 2},[990577],[153086]
1,00-0000,{'licitació de prova sobre digital 00/0000 - p...,{'79': 2},[884512],[283367],NaN,NaN,NaN,NaN,{'licitació de prova sobre digital 00/0000 - p...,{'79': 2},[884512],[283367]
2,000-expedient-2020-034,{'en el marc duna creixent proliferació quant ...,{'92': 2},[903809],[182156],NaN,NaN,NaN,NaN,{'en el marc duna creixent proliferació quant ...,{'92': 2},[903809],[182156]
3,000-expedient-2021-266,{'el present expedient té per objecte el submi...,{'34': 2},[972632],[157778],NaN,NaN,NaN,NaN,{'el present expedient té per objecte el submi...,{'34': 2},[972632],[157778]
4,00000057-2022,{'renovació de la xarxa daigua potable i impla...,{'45': 2},[1007173],[189410],NaN,NaN,NaN,NaN,{'renovació de la xarxa daigua potable i impla...,{'45': 2},[1007173],[189410]


In [ ]:
# # Unique indices (by id)
# _tot_id = len(matched_id) / len(valid_tend_cat)
# # Total elements (taking into account some values appear multiple times)
# _tot_id_sum = matched_id["title"].apply(lambda x: sum(x.values())).sum() / len(
#     valid_tend_cat
# )

# # Unique indices (by cpv and title)
# _tot_ti = len(matched_aux) / len(valid_tend_cat)
# # Total elements (taking into account some values appear multiple times)
# _tot_ti_sum = matched_aux["title"].apply(lambda x: sum(x.values())).sum() / len(
#     valid_tend_cat
# )

# # Unique aggregated
# _tot = len(matched_final) / len(valid_tend_cat)
# # Total elements (taking into account some values appear multiple times)
# _tot_sum = matched_final["title"].apply(lambda x: sum(x.values())).sum() / len(
#     valid_tend_cat
# )

# print(f"{'Unique indices (by id): ':<40}{_tot_id:.5f}")
# print(f"{'Total elements (by id): ':<40}{_tot_id_sum:.5f}")
# print()
# print(f"{'Unique indices (by cpv and title): ':<40}{_tot_ti:.5f}")
# print(f"{'Total elements (by cpv and title): ':<40}{_tot_ti_sum:.5f}")
# print()
# print(f"{'Unique indices (aggregated): ':<40}{_tot:.5f}")
# print(f"{'Total elements (aggregated): ':<40}{_tot_sum:.5f}")
